In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import networkx as nx

In [2]:
cb1 = pd.read_csv('cb1.csv')
cb2 = pd.read_csv('cb2.csv')
cb3 = pd.read_csv('cb3.csv')
cb = cb1.append(cb2)
cb = cb.append(cb3)
cb.index = range(len(cb))
cb['date_my'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('m')
cb['date_y'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('y')

nodes_org = cb[['target_name', 'target_ID', 'target_country', 'target_continent','industry']].drop_duplicates()
nodes_org['bipartite'] = 1
nodes_inv = cb[['investors_name', 'investor_id', 'investor_country', 'investor_continent', 'investor_type']].drop_duplicates()
nodes_org = nodes_org.rename(columns={"target_name": "node", "target_country": "country", 
                                      "target_continent": "continent", "industry":"industry",
                                     "target_ID":"index"})
nodes_inv = nodes_inv.rename(columns={"investors_name": "node","investor_country": "country", 
                                      "investor_continent": "continent", "investor_type":"industry","investor_id":'index' })
nodes_inv['bipartite'] = 0
nodes = nodes_org.append(nodes_inv, ignore_index= True)
nodes['id'] = nodes.index

numbers = cb.groupby(['target_name','stage'], as_index = False).count()[['target_name','stage','investors_name']].rename(columns={"investors_name": "investor_numbers"})
cb_new = pd.merge(cb, numbers,  how='left', left_on=['target_name','stage'], right_on = ['target_name','stage'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 1],  how='left', left_on=['target_name','target_ID'], right_on = ['node','index'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 0],  how='left', left_on=['investors_name','investor_id'], right_on = ['node','index'])
nodes = nodes.drop(columns = "index")


In [3]:
nodes

,node,country,continent,industry,bipartite,id
0,Shionogi & Co.,Japan,Asia,Pharmaceuticals / Drugs,1,0
1,Astellas Pharma,Japan,Asia,Pharmaceuticals / Drugs,1,1
2,Sumitomo Dainippon Pharma,Japan,Asia,Pharmaceuticals / Drugs,1,2
3,Konica Minolta,Japan,Asia,Medical Devices & Equipment,1,3
4,Takeda Pharmaceutical,Japan,Asia,Pharmaceuticals / Drugs,1,4
...,...,...,...,...,...,...
83253,Shenzhen Qianhai Beizeng Asset Management Co. ...,China,Asia,Venture Capital,0,83253
83254,Wuxi High-Tech District Seed Fund,China,Asia,Other,0,83254
83255,Edvantage Group,China,Asia,Corporation,0,83255
83256,Healgen,NaN,NaN,NaN,0,83256


In [6]:
nodes_fda = pd.read_csv('traject_3y.csv')
covariates = pd.read_csv(r'C:\Users\Marco\Documents\GitHub\crunch_net\ANS\ANS_all\Covariate_bidbid_final_final2.csv')
covariates_org = pd.read_csv(r'C:\Users\Marco\Documents\GitHub\crunch_net\ANS\ANS_all\Covariates_orgorg_final2.csv')

In [5]:
orgs = list(nodes_fda.columns)
orgs = [int(x) for x in orgs]

In [7]:
centralities = list(covariates.columns)
for i in range(0,len(centralities)):
    centralities[i] = centralities[i][:-4]
centralities = list(set(centralities))[1:]

In [8]:
centralities_org = list(covariates_org.columns)
for i in range(0,len(centralities_org)):
    centralities_org[i] = centralities_org[i][:-4]
centralities_org = list(set(centralities_org))[1:]

In [9]:
#CENTRALITITES OF ALL

sub0 = cb_new[['id_x','date_y','id_y', 'size_real']]
for cen in centralities: 
    locals()[cen+'_max'] = []
    locals()[cen+'_min'] = []
    locals()[cen+'_median'] = []
operators = ['_max', '_min', '_median']  
num_inv = []

s = 0

for cen_org in centralities_org:
    locals()[cen_org +'_org'] = []

for  i in tqdm(orgs):
    
    sub1 = sub0[sub0['id_x']==i]
    sub1 = sub1.dropna()
    sub2 = sub1[sub1['date_y']==sub1['date_y'].values[0]]
    anno = sub2['date_y'].values[0].year
    
    for z in ['inv', 'org']:
        if z == 'org':
            for cen_org in centralities_org:
                try:
                    locals()[cen_org + '_org'].append((covariates_org[covariates_org['id']== i][cen_org + str(anno)].values[0]))
                except:
                    locals()[cen_org + '_org'].append(np.nan)


        else:

            investors = set(sub2['id_y'])
            for cen in centralities:
                locals()[cen] = []
                for l in investors:
                    try:
                        locals()[cen].append((covariates[covariates['id']== l][cen + str(anno)].values[0]))
                    except:
                        locals()[cen].append(np.nan)
                locals()[cen+'_max'].append(np.nanmax(locals()[cen]))
                locals()[cen+'_min'].append(np.nanmin(locals()[cen]))
                locals()[cen+'_median'].append(np.nanmedian(locals()[cen]))



            num_inv.append(len(investors))
        
        s += 1
        if s % 500 == 0:
            print(s)

  0%|          | 0/7425 [00:00<?, ?it/s]

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15464\3412582721.py:42: RuntimeWarning: All-NaN axis encountered
  locals()[cen+'_max'].append(np.nanmax(locals()[cen]))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15464\3412582721.py:43: RuntimeWarning: All-NaN axis encountered
  locals()[cen+'_min'].append(np.nanmin(locals()[cen]))
c:\python38\lib\site-packages\numpy\lib\nanfunctions.py:1119: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500


In [10]:
covariate = nodes[(nodes['id'].isin(orgs))&(nodes['bipartite']==1)]

for cen in centralities:
    for op in operators:
        covariate[cen + op] = locals()[cen+op]
covariate['num_inv'] = num_inv

for cen_org in centralities_org:
    covariate[cen_org + '_org'] = locals()[cen_org + '_org']


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15464\1417526639.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covariate[cen + op] = locals()[cen+op]
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15464\1417526639.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covariate['num_inv'] = num_inv
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15464\1417526639.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [336]:
covariate.to_csv('covaria')

{'Alternative Medicine',
 'Biotechnology',
 'Dental Products & Services',
 'Disease Diagnosis',
 'Drug Delivery',
 'Drug Development',
 'Drug Discovery',
 'Drug Manufacturing',
 'Elective & Aesthetic Medicine',
 'Health & Wellness (non-internet/mobile)…',
 'Healthcare Plans',
 'Healthcare Software',
 'Internet Software & Services',
 'Medical Devices & Equipment',
 'Medical Equipment & Supplies',
 'Medical Facilities & Services',
 'Medical Information',
 'Medical Product Distribution',
 'Mobile Software & Services',
 'Pharmaceutical Distribution & Wholesale…',
 'Pharmaceuticals / Drugs',
 'Stealth Mode'}

In [5]:
def apply_function_network(lista_storage,lista_nomi,funzione,directed=False):
    for year in tqdm(range(1990,2019)):
        string='orgorg4_single'+str(year)+'.gpickle'
        network=nx.readwrite.read_gpickle(string)
        storage=str(funzione)+'_orgorg3_single'+str(year)
        lista_storage.append(funzione(network))
        lista_nomi.append(storage)
#         network=nx.readwrite.read_gpickle('bidbid_stage_self_cb'+str(year)+'.gpickle')
#         lista_storage.append(funzione(network))
#         lista_nomi.append(str(funzione)+'_bidder_'+str(year))
    return lista_storage,lista_nomi
def apply_function_network_community(lista_storage,lista_nomi,funzione,directed=False):
    for year in tqdm(range(1999,2020)):
        string='orgorg3_single_/orgorg3_single'+str(year)+'.gpickle'
        network=nx.readwrite.read_gpickle(string)
        storage='Louvain_orgorg3_single'+str(year)
        lista_storage.append(funzione(network))
        lista_nomi.append(storage)
    #network=nx.readwrite.read_gpickle('bidbid_stage_single_cb.gpickle')
    #lista_storage.append(funzione(network))
    #lista_nomi.append(str(funzione)+'_single')
    return lista_storage,lista_nomi

In [3]:
lista_funzioni_def=[nx.betweenness_centrality,nx.harmonic_centrality,nx.clustering, nx.core_number,nx.eigenvector_centrality, nx.degree,nx.closeness_centrality,nx.pagerank,nx.load_centrality,nx.average_neighbor_degree]



In [6]:
lista_storage,lista_nomi=apply_function_network(lista_storage,lista_nomi,nx.betweenness_centrality)

  0%|          | 0/29 [00:00<?, ?it/s]

In [4]:
# Array di funzioni da applicare - ciclo for su di esse
#lista_funzioni=[nx.voterank, nx.eigenvector_centrality, nx.degree_centrality, nx.pagerank, nx.closeness_centrality, nx.incremental_closeness_centrality, nx.load_centrality, nx.subgraph_centrality, nx.dispersion, nx.percolation_centrality,nx.average_neighbor_degree,nx.constraint, nx.betweenness_centrality]
# lista_funzioni2=[nx.dispersion,nx.constraint,nx.betweenness_centrality, nx.trophic_levels, nx.harmonic_centrality, nx.enumerate_all_cliques, nx.clustering, nx.core_number, nx.rich_club_coefficient, nx.closeness_vitality]
# lista_funzioni_rifare=[nx.harmonic_centrality, nx.enumerate_all_cliques, nx.clustering,nx.core_number,nx.closeness_vitality]
# Funzione prende df, se direzionato o meno
# lista_funzioni_prova2=[nx.eigenvector_centrality, nx.degree]
lista_storage=[]
lista_nomi=[]
for funzione in lista_funzioni_def:
    try:
        print('Trying'+str(funzione))
        lista_storage2, lista_nomi2=apply_function_network(lista_storage2,lista_nomi2,nx.betweenness_centrality)
    except:
        print(str(funzione)+' did not work')
        continue

Trying<function betweenness_centrality at 0x7fe0d0398dc0>
<function betweenness_centrality at 0x7fe0d0398dc0> did not work
Trying<function harmonic_centrality at 0x7fe0d03bc4c0>
<function harmonic_centrality at 0x7fe0d03bc4c0> did not work
Trying<function clustering at 0x7fe0989614c0>
<function clustering at 0x7fe0989614c0> did not work
Trying<function core_number at 0x7fe098973af0>
<function core_number at 0x7fe098973af0> did not work
Trying<function eigenvector_centrality at 0x7fe0d03bc3a0>
<function eigenvector_centrality at 0x7fe0d03bc3a0> did not work
Trying<function degree at 0x7fe10090ac10>
<function degree at 0x7fe10090ac10> did not work
Trying<function closeness_centrality at 0x7fe0d03a3a60>
<function closeness_centrality at 0x7fe0d03a3a60> did not work
Trying<function pagerank at 0x7fe1009fc310>
<function pagerank at 0x7fe1009fc310> did not work
Trying<function newman_betweenness_centrality at 0x7fe0d03c71f0>
<function newman_betweenness_centrality at 0x7fe0d03c71f0> did not 

In [4]:
network=nx.readwrite.read_gpickle('orgorg4_single2006.gpickle')

In [5]:
def apply_function_network(funzione):
    for year in tqdm(range(1990,2021)):
        string2='orgorg4_single'+str(year)+'.gpickle'
        network2=nx.readwrite.read_gpickle(string2)
        storage2=str(funzione)+'_orgorg4_single'+str(year)+'.csv'
        covariate2 = funzione(network2)
        covariate3 = pd.Series(covariate2)
        print('Arrivo qui', year)
        covariate3.to_csv(storage2)
        print('Done', funzione, 'for year', year)

In [ ]:
for function in lista_funzioni_def:
    apply_function_network(function)

  0%|          | 0/31 [00:00<?, ?it/s]

Arrivo qui 1990
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1990
Arrivo qui 1991
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1991
Arrivo qui 1992
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1992
Arrivo qui 1993
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1993
Arrivo qui 1994
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1994
Arrivo qui 1995
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1995
Arrivo qui 1996
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1996
Arrivo qui 1997
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1997
Arrivo qui 1998
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1998
Arrivo qui 1999
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1999
Arrivo qui 2000
Done <function betweenness_centrality at 0x7fa37894dee0> for year 2000
Arrivo qui 2001
Done <function betweenness_

In [9]:
import concurrent
with concurrent.futures.ProcessPoolExecutor() as executor:
    # results = [executor.submit(apply_function_network,funzione) for funzione in lista_funzioni]
    executor.map(apply_function_network, lista_funzioni_def)

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

In [7]:
lista_storage2=[]
lista_nomi2=[]
apply_function_network(lista_storage2,lista_nomi2,nx.degree)

NameError: name 'apply_function_network' is not defined

In [11]:
lista_nomi2[10]

'<function degree at 0x7fb1412bf9d0>_orgorg3_single2000'